In [ ]:
import pandas as pd 
import numpy as np
import scipy.stats as ss
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
# set some styling defaults for matplotlib
plt.style.use("seaborn-talk")
mpl.rcParams["figure.dpi"] = 90  # change this to set apparent figure size
mpl.rcParams["figure.figsize"] = (10, 4)
mpl.rcParams["figure.frameon"] = False

# set decimal precision to 3 dec. places
%precision 3

# Practical task: Bayesian product design

https://app.sli.do/event/rDUA5XYTvm9RmYtLHAQit9

## What's the goal?

### A Bayesian food thermometer

<img src="imgs/grill.jpg" width="70%">

**[Image by Ahmed Ardity: https://pixabay.com/photos/food-temperature-safety-hot-cook-5053725/]**

We'll explore how Bayesian ideas could be used in product design for everyday devices. We'll build a **Bayesian meat thermometer** (or food thermometer, if you are vegetarian).

Meat thermometers are essential tools in correctly cooking meats and fish (they're also essential for making candy). They are especially useful with unpredictable heating sources like barbecues. It's dangerous to eat undercooked food, and some foods (like shellfish) become unpleasant rapidly if overcooked. Chefs use thermometers to ensure their food is safe and tasty.

### The problem

<img src="imgs/thermometer_meat.png" width="65%">

What we want to know is how well cooked the food is (the hidden state). What we observe is a digitised voltage from a temperature sensor. Unfortunately, we bought cheap temperature sensors, which are slow to react and quite noisy. Standard thermometers just display a calibrated version of the sensor voltage on an LCD: the raw sensor value. 

Problems:

* It takes quite some time for the temperature reading to stabilise
* The readings aren't accurate (mis-calibration) or precise (noise).
* We don't know how far into the food the thermometer has been pushed. Deeper regions are likely to be less cooked.
* The true temperature is continuously changing
* We want a goodness of cooking, not a temperature

> Caveat: We don't really need a Bayesian meat thermometer -- careful use of a standard thermometer works just fine. But we'll explore how Bayesian representations could be used in a product like this.

<img src="imgs/temperature_problem.png" width="95%">

### The steps
We'll look at three aspects of this design in the next hour. In Part I, you'll apply Bayesian optimisation to select a smoothing parameter for the temperature sensor. In Part II, you'll perform Bayesian inference to infer the cookedness, and display it to the user. In Part III, you'll analyse how well you did in a Bayesian analysis.

## How should you approach this?

* There's a skeleton *non-Bayesian* meat thermometer working and implemented
* There are helper functions that can be plugged in to add Bayesian super powers


### Thermometer simulator

### Controls
* When you use the thermometer, you have the following controls:

    * **Cook 10, 5, 1**: cook the food for another 10, 5 or 1 minutes
    * **In**: move the thermometer in a bit
    * **Out**: move the thermometer out a bit    
    * **Serve**: take the food and out and serve it
    
* The display updates continuously.

### Cookedness
<img src="imgs/chicken_curve.png">

### Metrics
The metric is the best cooked food with the least time spent looking at the display. The cookedness will automatically be computed for you. You'll get a single overall score that combines time and cookedness.

## First steps

### Establishing a baseline
Use the baseline thermometer, and cook five dishes. Record your scores. Note: dishes are not always the same!


### Planning your time

## Part I: optimising the sensor

## Part II: Bayesian interaction and display

## Part III: Bayesian analysis






In [ ]:
# times in minutes
# radius in cm
# temps in *c

meat_temps = {"salmon":(49,60) , "steak":(55, 70), "shrimp":(61, 64), "chicken":(71, 90), "pork":(71, 85), "boar":(76, 81)}

def meat_utility(temps, mn, mx):
    rnge = (mx-mn)
    ctr = (mx+mn)/2
    utility = np.where(temps<mn, -1*((mn-temps)**2), np.where(temps>mx, -0.45/rnge*((temps-mx)**2), 1-0.01*(temps-ctr)**2))
    return utility
    
    
    
def plot_chicken_curve(foods):
    fig, ax = plt.subplots()
    temps = np.linspace(40, 100, 200)
    mn, mx = foods["chicken"]    
    utility = meat_utility(temps, mn, mx)
    ax.plot(temps, np.exp(utility/10.0), 'k')
    ax.axvline(65, ls=':', c='k')
    ax.axvline(70, ls=':', c='k')
    ax.axvline(80, ls=':', c='k')
    ax.axvline(90, ls=':', c='k')
    ax.text(60, 0.8, 'Dangerous', ha='center')
    ax.text(68, 0.8, 'Gooey', ha='center')
    ax.text(75, 0.8, 'Well cooked', ha='center')
    ax.text(85, 0.8, 'Overdone', ha='center')
    ax.text(95, 0.8, 'Burned', ha='center')
    
    ax.set_xlabel("Temp. C")
    ax.set_ylabel("Goodness")
    ax.set_title("Chicken internal temperature")
    
plot_chicken_curve(meat_temps)
plt.savefig("imgs/chicken_curve.png", bbox_inches="tight")
    
def plot_utility_curves(foods):
    fig, ax = plt.subplots()
    temps = np.linspace(30, 100, 200)
    for food, (mn, mx) in foods.items():
        utility = meat_utility(temps, mn, mx)
        ax.plot(temps, np.exp(utility/10.0), label=food)
    ax.legend()
    ax.set_xlabel("Temp. C")
    ax.set_ylabel("Goodness")

plot_utility_curves(meat_temps)


In [ ]:
                  
                   
        

def get_mass(radius, density):
    return radius ** 2.5 * density

def thermometer_model(radius, insertion_depth, init_temp, cooking_time, cooking_temp, watch_time, density, heat_capacity, noise_level):
    
    def food_model():        
        mass = get_mass(radius, density)
        alpha = 1.0/((mass * heat_capacity) * (insertion_depth+radius/2)+1+np.random.normal(0, 0.001))            
        temp = cooking_temp + (init_temp - cooking_temp) * np.exp(-alpha * cooking_time)
        
        temp = np.tanh(temp/120) * 120
        return temp
    
    def thermometer_model(real_temp):
        beta = 5 + np.random.normal(0, 0.2)
        temp = real_temp + (init_temp - real_temp) * np.exp(-beta * watch_time)
        temp = temp + np.random.normal(0, noise_level, temp.shape)
        return temp
    
    return thermometer_model(food_model())

print(get_mass(10, 0.9))
fig, ax = plt.subplots()
times = np.linspace(0, 100, 200)
t = thermometer_model(10, 0.0, 20.0, times, 180.0, 1.0, 0.9, 0.02, 1)
ax.plot(times, t)
    
    
import random    
def cook():
    radius = np.random.uniform(9, 25)
    room_temp = 20
    density = 0.9
    heat_capacity = 0.01
    oven_temp = np.random.randint(0, 5) * 10 + 150
    meat = random.choice(list(meat_temps.keys()))
    mass = get_mass(radius, density)
    print(f"You are cooking {mass/1000:.1f}kg of {meat} in an oven at {oven_temp}C")
    for i in range(100):
        print(f"Time = {i} minutes")
        t = thermometer_model(radius, radius, room_temp, i, oven_temp, 1000.0, density, heat_capacity, 0.0)
        print(f"True internal temperature = {t:.0f}C")
        t = thermometer_model(radius, 0, room_temp, i, oven_temp, 1000.0, density, heat_capacity, 0.0)
        print(f"True surface temperature = {t:.0f}C")
        t = thermometer_model(radius, radius/2, room_temp, i, oven_temp, 0.5, density, heat_capacity, 0.1)
        print(f"Thermometer temperature at half-way = {t:.0f}C")
        
    
cook()
    